# Homework exercise 1
## Deadline: upload to Moodle by 19 November 18:00 h

__Suggestion: take this notebook and simply add your code and explanations. Your submission needs to include your code's output (even if the code throws an error).__

If you prefer to use .py files, you are expected to also include a PDF containing the output of your code and your explanations. Still, the code needs to be in a form that can be easily run on another computer.

If you use any file paths, assign paths to variables at the beginning of your code and use those variables when later referring to the paths.

__Name :__ Moritz-Jakob Leithner


The name of the file that you upload should be named *Homework1_YourLastName_YourStudentID*.

Reminder: you are required to attend class on 20 November to earn points for this homework exercise unless you have a valid reason for your absence.

You are encouraged to work on this exercise in teams of up to three students. If any part of the questions is unclear, please ask on the Moodle forum.

#### Simulating a stock market
Consider a financial market consisting of one or more stock exchanges. In this market, only one asset is traded (to keep it simple). 

Each stock exchange operates as a limit order book. I.e., if an investor Alice submits, e.g., a buy order, she specifies the maximum price she is willing to pay (called the limit price) and the number of shares she would like to trade (we will set the number of shares to 1 for simplicity). If there already is an existing sell order in the order book (previously submitted by another trader we will call Bob) with a limit price below or equal to Alice's limit price, a trade happens at Bob's limit price. If there is no such existing order, Alice's order enters the limit order book to wait for a seller who might be willing to trade at her limit price. Assume, for simplicity and contrary to real markets, that an order cannot be cancelled or modified after it has been submitted.

This problem set consists of tasks asking you to
* implement the functionality of a stock exchange operating as a limit order market
* generate random data representing orders submitted to the market
* simulate a market consisting of one or more stock exchanges using the randomly generated order flow
* finally interpret the output generated by your code

__You are expected to implement your solution using basic Python functionality plus numpy (in particular, for the generation of random numbers).__

1. Implementing a limit oder market

You are strongly encouraged to implement the limit order market as a class, so that you can later easily generate exchanges with slightly varying parameters.

The limit order market needs to offer the following functionality:

* access to the best bid and the best ask price (as attributes of the market or via methods)
* processing of incoming orders, where orders have the parameters `limit_price`, `side` (i.e. buy or sell), `size` (i.e. number of shares), and an `order_id` (i.e. an integer identifying an order). Processing constitutes of 
    * updating the order book (i.e. inserting the new order if it does not trade immediately, otherwise remove or modify the order against which the new order trades)
    * returning information on whether a trade happened and, if so, return trade price, size, and the IDs of the order that traded against one another
    
The only parameter pertaining to the order book is the tick size, which is a measure of the granularity of the prices the exchange allows. E.g., a tick size of 0.01 means that limit prices must be numbers expressed in whole cents.     

__Note:__ if there are multiple orders that would match with the new order, the buy order with the highest limit price/sell order with the lowest limit price will trade against the new order. If multiple orders at the same price exist in the order book, the order submitted the earliest get to trade first. This rule is called price-time priority and is the most common, though not the only, rule applied in real limit order books.

In [23]:
from datetime import datetime
import numpy as np
import pandas as pd

import bisect

class Order(object):
    number = 0

    def __init__(self,limit_price, timestamp = None):
        self.limit_price = limit_price
        self.timestamp = timestamp if timestamp is not None else datetime.now()
        self.ID = Order.number + 1
        Order.number += 1
        #print('Order (ID: %s) created' %(self.ID))
    
    def __str__(self):
        return " Limit Price: %s, Side: %s, Order ID: %s, Timestamp: %s" %(self.limit_price, self.side, self.ID, self.timestamp)

    def __del__(self):
        #print('Order (ID: %s) deleted' %(self.ID))
        pass

class Ask(Order):

        def __init__(self, limit_price, size = 1, side = 'sell', timestamp = None):
            timestamp = timestamp if timestamp is not None else datetime.now()

            Order.__init__(self, limit_price, timestamp)
            self.limit_price = limit_price
            self.size = size
            self.side = side


class Bid(Order):

        def __init__(self, limit_price, size = 1, side = 'buy', timestamp = None):
            timestamp = timestamp if timestamp is not None else datetime.now()

            Order.__init__(self, limit_price, timestamp)
            self.limit_price = limit_price
            self.size = size
            self.side = side


class OrderBook(object):
    def __init__(self, tick_size):
        self.tick_size = tick_size
        self.asks = []
        self.bids = []

        self.price_changes = []
        self.total_volume = 0
        self.spreads = []       #list of all spreads from Trades in OrderBook To calculate avg. spread
        self.hist_spreads = []  #list of the historical average spreads
        self.hist_std = []
        self.trade_prices = []


    def add_order(self, order):
        timestamp = datetime.now()   #when do we ever need this?
        order_key = (order.limit_price, order.timestamp)

        if order.side == 'sell':

            pos = bisect.bisect_right([(ask.limit_price, ask.timestamp) for ask in self.asks], order_key)
            self.asks.insert(pos, order)

            if not self.bids or order.limit_price <= self.get_best_bid():
                self.match_order(order, self.bids)
  
        elif order.side == 'buy':

            pos = bisect.bisect_left([(bid.limit_price, bid.timestamp) for bid in self.bids], order_key)
            self.bids.insert(pos, order)

            if not self.asks or order.limit_price >= self.get_best_ask():
                self.match_order(order, self.asks)

    def is_match(self, new_order, existing_order):
        if new_order.side == 'sell' and existing_order.side == 'buy':
            return new_order.limit_price <= existing_order.limit_price
        
        elif new_order.side == 'buy' and existing_order.side == 'sell':
            return new_order.limit_price >= existing_order.limit_price
        else:
            return False

    def match_order(self, new_order, orders):
        for index, order in enumerate(orders):
            if self.is_match(new_order, order):

                bid_price = new_order.limit_price if new_order.side == 'buy' else order.limit_price
                ask_price = new_order.limit_price if new_order.side == 'sell' else order.limit_price

                trade = Trade(new_order.ID if new_order.side == 'buy' else order.ID, order.ID if new_order.side =='buy' else new_order.ID, bid_price, ask_price, order.limit_price, 1)
                #print("Trade executed: ", trade)

                self.update_trade_metrics(trade)

                orders.pop(index)

                if new_order.side == 'sell':
                    self.asks = [ask for ask in self.asks if ask.ID != new_order.ID]
                else:
                    self.bids = [bid for bid in self.bids if bid.ID != new_order.ID]

                break
    
    def update_trade_metrics(self, trade):
        self.total_volume += trade.size
        self.spreads.append(trade.bid_price - trade.ask_price)

        average_spread = self.get_average_spread()
        self.hist_spreads.append(average_spread)

        if self.trade_prices:
            percentage_change = (trade.price / self.trade_prices[-1]) - 1
            self.price_changes.append(percentage_change)

        std_price_change = self.get_std_dp()
        self.hist_std.append(std_price_change)

        self.trade_prices.append(trade.price)
    
    def get_average_spread(self):
        if not self.spreads:
            return 0
        return sum(self.spreads) / len(self.spreads)
    
    def get_std_dp(self):
        if len(self.price_changes) < 2:
            return 0
        return np.std(self.price_changes)
    

    def get_best_ask(self):
        if not self.asks:
            return None
            
        best_ask = min(self.asks, key = lambda x: x.limit_price)
        return best_ask.limit_price

    def get_best_bid(self):
        if not self.bids:
            return None

        best_bid = max(self.bids, key = lambda x: x.limit_price)
        return best_bid.limit_price     

        
    def display(self):
        print("Total number of orders: %s" % Order.number)
        print("Max bid: %s" % self.get_best_bid())
        print("Min ask: %s" % self.get_best_ask())
        print("Bids: ")
        for bid in self.bids:
            print(bid)
        print("Asks: ")
        for ask in self.asks:
            print(ask)
        

class Trade(object):
    number_of_trades = 0

    def __init__(self, buy_order_id, sell_order_id, bid_price, ask_price, price, size):
        self.buy_order_id = buy_order_id
        self.sell_order_id = sell_order_id
        self.bid_price = bid_price
        self.ask_price = ask_price
        self.price = price
        self.size = size
        Trade.number_of_trades += 1

    def __str__(self):
        return "(%s) : Buy Order ID: %s, Sell Order ID: %s, Price: %s/%s -> %s, Size: %s" % (Trade.number_of_trades, self.buy_order_id, self.sell_order_id, self.bid_price, self.ask_price, self.price, self.size)


2. Generating random numbers and simulating order flow

You are asked to allow the orders submitted to the market to depend on the following parameters:

* the fair value of the security
* a private value an individual trader experiences from holding the asset. This private value reflect how much the trader likes or dislikes owning the asset

We are going to assume the following simple rule determining side and limit price: if the private value is positive (negative), the trader will submit a buy (sell) order at a price equal to the sum of fair value and private value minus (plus) a constant `k`, which we will interpret as the required "profit", rounded down (up) to the nearest tick. This rule is not intended to reflect optimal behavior by the traders but rather serves to get a relatively reasonable simulation of the trading process. 

Write a function that generates the orders based on the parameters described above. Generate 100,000 random values following a standard normal distribution. Also

* generate 100,000 random numbers following the uniform distribution between 0 and 1, which we will use to determine changes to the fair value in the next step
* generate 100,000 binary random numbers with equal probability, which we will use to determine whether any change to the fair value is positive or negative
* generate 100,000 binary random numbers with equal probability, which we will use to determine the exchange to which an order is submitted if the trader is indifferent in the setting with two exchanges

In [52]:
def mk_order(fv, pv, k, exchange1, exchange2):
    '''individual orders based on params:
    - fv: fair value
    - pv: private value
    - k: profit constant
    - exchange1: the first exchange order book
    - exchange2: the second exchange order book
    '''

    limit_price = ((fv + pv - k) / exchange1.tick_size)//1 * exchange1.tick_size if pv > 0 else ((fv + pv + k) / exchange2.tick_size)//1 * exchange2.tick_size
    order_side = 'buy' if pv > 0 else 'sell'
    order = Bid(limit_price) if order_side == 'buy' else Ask(limit_price)

    chosen_exchange = None

    if order_side == 'sell':
        best_bid_e1 = exchange1.get_best_bid()
        best_bid_e2 = exchange2.get_best_bid()

        if best_bid_e1 is not None and best_bid_e2 is not None:
            chosen_exchange = exchange1 if best_bid_e1 > best_bid_e2 else exchange2
        elif best_bid_e1 is not None and limit_price <= best_bid_e1:
            chosen_exchange = exchange1
        elif best_bid_e2 is not None and limit_price <= best_bid_e2:
            chosen_exchange = exchange2
    else:  #BUY
        best_ask_e1 = exchange1.get_best_ask()
        best_ask_e2 = exchange2.get_best_ask()
        
        if best_ask_e1 is not None and best_ask_e2 is not None:
            chosen_exchange = exchange1 if best_ask_e1 < best_ask_e2 else exchange2
        elif best_ask_e1 is not None and limit_price >= best_ask_e1:
            chosen_exchange = exchange1
        elif best_ask_e2 is not None and limit_price >= best_ask_e2:
            chosen_exchange = exchange2

    if chosen_exchange is None:
        chosen_exchange = rng.choice([exchange1, exchange2])

    chosen_exchange.add_order(order)

    return order, chosen_exchange



#Set up for simulations
tick = [0.01, 0.1, 1]
Q = [0,0.001, 0.01, 0.1]
K = [0.1, 1]
fair_value = 100


exchange1 = OrderBook(tick[2])
exchange2 = OrderBook(tick[1])


rng = np.random.default_rng(1234)

A = rng.standard_normal(10000)
A1 = 0.1 * A
PrV = [A, A1]


B = rng.uniform(0, 1, 10000)
C = rng.choice([-1,1], 10000)


3. Simulating the market

Based on what you implemented in the previous steps, please simulate a market for all combinations of the following parameters:

* Initital fair value: 100
* Constant size for all orders: 1
* Probability of an increase/decrease of the fair value by 1 in each period (1 order = 1 period): 0, 0.001, 0.01, 0.1
* Parameter `k` (see above): 0.1, 1
* Standard deviation of private values: 0.1, 1
* Exchanges: 
    * one exchange with tick size 0.01, 0.1, 1
    * two exchanges each with tick size 0.01, 0.1, 1
    * one exchange with tick size 0.01, another with tick size 0.1 
    * one exchange with tick size 0.1, another with tick size 1 

Rules for order submission with multiple exchanges:

* If the order can immediately trade on one of the two exchanges, send the order to that exchange
* If the order can immediately trade on neither of the exchanges, send the order with equal probability to either of the exchanges
* If the order can immediately trade on both exchanges, send the order to the exchange offering the better price

For each simulation, compute the total trading volume, the average bid-ask spread (simple difference between ask and bid) on each exchange, and the standard deviation of changes in trade prices. For simulations containing two exchanges, also compute the trading volumes on each exchange.

In [57]:
simulation1 = []  

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[0])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange1)

            total_vol = exchange1.total_volume + exchange2.total_volume

            simulation1.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'avg_spread': exchange1.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp()
            })


In [ ]:
df1 = pd.DataFrame(simulation1)
print(df1)

In [59]:
simulation2 = []  # Define the simulation_results list

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[1])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange1)

            total_vol = exchange1.total_volume + exchange2.total_volume

            simulation2.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'avg_spread': exchange1.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp()
            })


In [60]:
df2 = pd.DataFrame(simulation2)
print(df2)

      k      q  total_vol  tick1  avg_spread  sigma_price_change1
0   0.1  0.000       8047    0.1    1.407292             0.007277
1   0.1  0.001       8045    0.1    1.399759             0.007470
2   0.1  0.010       8047    0.1    1.384411             0.007979
3   0.1  0.100       8042    0.1    1.606395             0.011396
4   1.0  0.000       6227    0.1    0.165009             0.006358
5   1.0  0.001       6925    0.1    0.166390             0.007648
6   1.0  0.010       7492    0.1    0.658942             0.016810
7   1.0  0.100       7703    0.1    1.457186             0.010868
8   0.1  0.000       6274    0.1    0.014727             0.000709
9   0.1  0.001       7121    0.1    0.353825             0.005745
10  0.1  0.010       7640    0.1    0.776745             0.014382
11  0.1  0.100       7779    0.1    1.422251             0.010558
12  1.0  0.000       3069    0.1    0.000000             0.000000
13  1.0  0.001       3960    0.1    0.135690             0.000338
14  1.0  0

4. Try to interpret the results computed above. How do trading volume, price volatility, and (for the setting with two exchanges) the proportion of trades on each exchange depend on the parameters of the simulation?

In [61]:
simulation3 = []  

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[2])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange1)

            simulation3.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'avg_spread': exchange1.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp()
            })


In [62]:
df3 = pd.DataFrame(simulation3)
print(df3)

      k      q  total_vol  tick1  avg_spread  sigma_price_change1
0   0.1  0.000       8047      1    1.461832             0.007904
1   0.1  0.001       8047      1    1.448775             0.008180
2   0.1  0.010       8047      1    1.434914             0.008654
3   0.1  0.100       8044      1    1.640201             0.011451
4   1.0  0.000       6274      1    0.147270             0.007127
5   1.0  0.001       7164      1    0.212454             0.007928
6   1.0  0.010       7683      1    0.497399             0.015218
7   1.0  0.100       7806      1    1.063120             0.011497
8   0.1  0.000       6274      1    0.000000             0.007118
9   0.1  0.001       7121      1    0.117966             0.007317
10  0.1  0.010       7641      1    0.471129             0.014352
11  0.1  0.100       7782      1    1.093571             0.010739
12  1.0  0.000       3069      1    0.000000             0.000000
13  1.0  0.001       5514      1    0.285072             0.000461
14  1.0  0

In [63]:
simulation4 = []  # Define the simulation_results list

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[0])
            exchange2 = OrderBook(tick[0])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange2)


            simulation4.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'vol1': exchange1.total_volume,
                'vol2': exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'tick2':exchange2.tick_size,
                'avg_spread1': exchange1.get_average_spread(),
                'avg_spread2': exchange2.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp(),
                'sigma_price_change2': exchange2.get_std_dp()
            })


In [64]:
df4 = pd.DataFrame(simulation4)
print(df4)

      k      q  total_vol  tick1  tick2  avg_spread1  avg_spread2  \
0   0.1  0.000       4978   0.01   0.01     1.414286     1.396255   
1   0.1  0.001       4976   0.01   0.01     1.405492     1.389487   
2   0.1  0.010       4978   0.01   0.01     1.423713     1.356392   
3   0.1  0.100       4941   0.01   0.01     1.725415     1.874571   
4   1.0  0.000       3126   0.01   0.01     0.288333     0.170272   
5   1.0  0.001       3826   0.01   0.01     0.205714     0.174527   
6   1.0  0.010       4358   0.01   0.01     0.649755     1.916064   
7   1.0  0.100       4575   0.01   0.01     1.300399     2.800458   
8   0.1  0.000       3156   0.01   0.01     0.017564     0.016434   
9   0.1  0.001       4009   0.01   0.01     0.199333     0.366569   
10  0.1  0.010       4519   0.01   0.01     1.371385     0.850880   
11  0.1  0.100       4622   0.01   0.01     1.432477     1.758417   
12  1.0  0.000          0   0.01   0.01     0.000000     0.000000   
13  1.0  0.001        891   0.01  

In [65]:
simulation5 = []  

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[1])
            exchange2 = OrderBook(tick[1])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange2)

            total_vol = exchange1.total_volume + exchange2.total_volume

            simulation5.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'vol1': exchange1.total_volume,
                'vol2': exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'tick2':exchange2.tick_size,
                'avg_spread1': exchange1.get_average_spread(),
                'avg_spread2': exchange2.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp(),
                'sigma_price_change2': exchange2.get_std_dp()
            })


In [66]:
df5 = pd.DataFrame(simulation5)
print(df5)

      k      q  total_vol  tick1  tick2  avg_spread1  avg_spread2  \
0   0.1  0.000       4978    0.1    0.1     1.410071     1.404740   
1   0.1  0.001       4976    0.1    0.1     1.401572     1.397862   
2   0.1  0.010       4978    0.1    0.1     1.369550     1.404196   
3   0.1  0.100       4951    0.1    0.1     1.748545     1.831012   
4   1.0  0.000       3156    0.1    0.1     0.211429     0.165428   
5   1.0  0.001       3853    0.1    0.1     0.139170     0.180327   
6   1.0  0.010       4408    0.1    0.1     0.971571     0.632252   
7   1.0  0.100       4588    0.1    0.1     1.461273     1.586916   
8   0.1  0.000       3205    0.1    0.1     0.000000     0.014755   
9   0.1  0.001       4051    0.1    0.1     0.422222     0.354033   
10  0.1  0.010       4508    0.1    0.1     3.825146     0.721490   
11  0.1  0.100       4647    0.1    0.1     1.828353     1.563634   
12  1.0  0.000          0    0.1    0.1     0.000000     0.000000   
13  1.0  0.001        891    0.1  

In [67]:
simulation6 = [] 

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[2])
            exchange2 = OrderBook(tick[2])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange2)

            total_vol = exchange1.total_volume + exchange2.total_volume

            simulation6.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'vol1': exchange1.total_volume,
                'vol2': exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'tick2':exchange2.tick_size,
                'avg_spread1': exchange1.get_average_spread(),
                'avg_spread2': exchange2.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp(),
                'sigma_price_change2': exchange2.get_std_dp()
            })


In [68]:
df6 = pd.DataFrame(simulation6)
print(df6)

      k      q  total_vol  tick1  tick2  avg_spread1  avg_spread2  \
0   0.1  0.000       4978      1      1     1.487064     1.437672   
1   0.1  0.001       4978      1      1     1.450509     1.447087   
2   0.1  0.010       4978      1      1     1.441813     1.432930   
3   0.1  0.100       4957      1      1     1.780329     1.763552   
4   1.0  0.000       3203      1      1     0.000000     0.148171   
5   1.0  0.001       4093      1      1     0.307692     0.212745   
6   1.0  0.010       4549      1      1     4.417266     0.447166   
7   1.0  0.100       4703      1      1     0.890613     1.250146   
8   0.1  0.000       3204      1      1     0.000000     0.000313   
9   0.1  0.001       4049      1      1     0.272727     0.118375   
10  0.1  0.010       4550      1      1     2.057554     0.390918   
11  0.1  0.100       4655      1      1     1.512580     1.039157   
12  1.0  0.000          0      1      1     0.000000     0.000000   
13  1.0  0.001       2445      1  

In [69]:
simulation7 = []  

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[0])
            exchange2 = OrderBook(tick[1])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange2)

            total_vol = exchange1.total_volume + exchange2.total_volume

            simulation7.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'vol1': exchange1.total_volume,
                'vol2': exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'tick2':exchange2.tick_size,
                'avg_spread1': exchange1.get_average_spread(),
                'avg_spread2': exchange2.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp(),
                'sigma_price_change2': exchange2.get_std_dp()
            })


In [70]:
df7 = pd.DataFrame(simulation7)
print(df7)

      k      q  total_vol  tick1  tick2  avg_spread1  avg_spread2  \
0   0.1  0.000       4978   0.01    0.1     1.437678     1.464521   
1   0.1  0.001       4976   0.01    0.1     1.433277     1.454341   
2   0.1  0.010       4978   0.01    0.1     1.424722     1.468070   
3   0.1  0.100       4954   0.01    0.1     1.749162     2.022608   
4   1.0  0.000       3152   0.01    0.1     0.295000     0.200995   
5   1.0  0.001       3840   0.01    0.1     0.181200     0.208936   
6   1.0  0.010       4419   0.01    0.1     1.703077     0.692194   
7   1.0  0.100       4589   0.01    0.1     1.253290     2.678176   
8   0.1  0.000       3205   0.01    0.1     0.045714     0.039350   
9   0.1  0.001       4048   0.01    0.1     0.397778     0.386459   
10  0.1  0.010       4561   0.01    0.1     1.365472     0.796376   
11  0.1  0.100       4675   0.01    0.1     1.371353     1.557530   
12  1.0  0.000          0   0.01    0.1     0.000000     0.000000   
13  1.0  0.001        891   0.01  

In [71]:
simulation8 = []  

for A in PrV:
    for k in K:
        for q in Q:
            
            exchange1 = OrderBook(tick[2])
            exchange2 = OrderBook(tick[1])
            fair_value = 100

            for pv, prchange, dirchange in zip(A, B, C):
                if prchange < q:
                    fair_value += dirchange
                else:
                    fair_value += dirchange * 0
                order = mk_order(fair_value, pv, k, exchange1, exchange2)

            total_vol = exchange1.total_volume + exchange2.total_volume

            simulation8.append({
                'k': k,
                'q': q,
                'total_vol': exchange1.total_volume + exchange2.total_volume,
                'vol1': exchange1.total_volume,
                'vol2': exchange2.total_volume,
                'tick1': exchange1.tick_size,
                'tick2':exchange2.tick_size,
                'avg_spread1': exchange1.get_average_spread(),
                'avg_spread2': exchange2.get_average_spread(),
                'sigma_price_change1': exchange1.get_std_dp(),
                'sigma_price_change2': exchange2.get_std_dp()
            })


In [72]:
df8 = pd.DataFrame(simulation8)
print(df8)

      k      q  total_vol  tick1  tick2  avg_spread1  avg_spread2  \
0   0.1  0.000       4976      1    0.1     0.975454     0.993025   
1   0.1  0.001       4975      1    0.1     0.949980     1.003858   
2   0.1  0.010       4976      1    0.1     0.994272     1.026336   
3   0.1  0.100       4925      1    0.1     1.423713     1.591828   
4   1.0  0.000       1893      1    0.1     0.346154     0.455053   
5   1.0  0.001       2512      1    0.1     0.775000     0.466746   
6   1.0  0.010       3743      1    0.1     2.368929     1.105891   
7   1.0  0.100       4356      1    0.1     1.686004     2.409258   
8   0.1  0.000       1609      1    0.1     0.100000     0.113246   
9   0.1  0.001       3244      1    0.1     0.034169     0.358289   
10  0.1  0.010       4165      1    0.1     0.932626     0.892346   
11  0.1  0.100       4496      1    0.1     1.624152     1.874229   
12  1.0  0.000          0      1    0.1     0.000000     0.000000   
13  1.0  0.001        891      1  